In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
stations = sorted(glob.glob('raw_data/*_stations*'))
rain_data = sorted(glob.glob('raw_data/*_precipitation*'))

dates = np.array([i.split('/')[-1].split('_')[0] for i in stations])

*TO-DO*
- [x] Obtain a master reference for all stations that are present within the dataset
- [x] Function to arrange rainfall must
    1. Be able to have data for all dates present in the raw data folder in one dataframe
    2. Be able to create a dataframe that isn't dependant on having the same stations across all source data

In [3]:
def concat_stations(station_ids):
    # n = station_ids.size
    # for i in range(0, n-1):
    #     df = pd.concat([station_ids[i], station_ids[i+1]], ignore_index=True)

    # df.drop_duplicates(subset = 'device_id', ignore_index = True, inplace = True)

    d = {}
    x = len(station_ids)
    for i in range(x):
        curr_df = station_ids[i]
        for j in range(curr_df.shape[0]):
            curr_row = curr_df.iloc[j]
            d_id = curr_row['device_id']
            if d_id in d:
                continue
            else:
                d[d_id] = curr_row

    df = pd.DataFrame(d).T

    return df

def collate_data(all_stations, sum_preci, dates):
    row = dates.size
    column_id = np.array(all_stations['device_id'])

    df = pd.DataFrame(np.zeros((row, column_id.size)), columns = column_id, index = dates)

    for i, series in enumerate(sum_preci):
        cols = series.columns
        vals = series.to_numpy()
        df.loc[dates[i], cols] = vals[0]

    return df 

In [4]:
station_ids = np.zeros(len(stations), dtype = object)
sum_rain = np.zeros(len(stations), dtype = object)

for i in range(len(stations)):
    station = pd.read_csv(stations[i])
    station = station.drop(['Unnamed: 0', 'id'], axis = 1)
    station = station.astype({'longitude':'float','latitude':'float'})
    
    rain = pd.read_csv(rain_data[i])
    times = rain['Time'].to_numpy()
    rain = rain.drop(['Time'], axis = 1)
    rain = rain.astype(float)
    
    station_ids[i] = station
    sum_rain[i] = pd.DataFrame(rain.sum()).T

all_stations = concat_stations(station_ids)
all_rain = collate_data(all_stations, sum_rain, dates)

In [5]:
all_rain.to_csv('./collated_daily_data.csv')
all_stations.to_csv('./all_stations.csv')

In [6]:
all_rain

,S77,S109,S117,S55,S64,S90,S61,S114,S11,S50,...,S222,S223,S224,S226,S227,S228,S229,S230,S205,S113.1
2017-03-01,34.0,38.0,10.0,42.8,33.0,25.6,32.6,25.2,15.0,36.8,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.000,NaN
2017-03-06,20.8,17.8,4.8,11.2,3.2,8.8,40.4,9.0,4.0,5.4,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.000,NaN
2017-03-11,8.6,0.2,0.0,4.0,28.2,1.6,0.0,40.8,59.6,12.8,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.000,NaN
2017-03-16,0.0,0.0,1.0,0.0,8.0,0.0,0.0,4.6,3.2,1.2,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.000,NaN
2017-03-21,0.0,13.4,0.0,5.6,0.0,3.4,5.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-29,16.6,21.6,0.0,0.0,0.0,14.8,0.0,21.0,0.0,19.4,...,16.884,13.195,19.497,15.0,39.592,6.732,10.556,13.0,8.772,NaN
2022-08-03,5.2,17.0,0.0,0.0,0.0,18.6,0.0,43.0,0.0,26.4,...,5.628,11.165,0.000,11.4,5.454,24.948,58.464,16.2,11.016,NaN
2022-08-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.402,0.0,0.000,0.000,0.000,0.0,0.000,NaN
2022-08-13,3.2,17.0,0.0,0.0,0.0,3.4,0.0,15.4,0.0,13.4,...,0.804,2.030,13.266,9.4,6.666,41.778,53.592,4.8,4.080,NaN


In [7]:
all_stations

,device_id,name,longitude,latitude
S77,S77,Alexandra Road,103.8125,1.2937
S109,S109,Ang Mo Kio Avenue 5,103.8492,1.3764
S117,S117,Banyan Road,103.679,1.256
S55,S55,Buangkok Green,103.886,1.3837
S64,S64,Bukit Panjang Road,103.7603,1.3824
...,...,...,...,...
S227,S227,S227,103.80389,1.43944
S228,S228,S228,103.70073,1.34703
S229,S229,S229,103.72195,1.35167
S230,S230,S230,103.76444,1.30167


In [8]:
# station_ids[0]
def concat_stations(station_ids):
    # n = station_ids.size
    # for i in range(0, n-1):
    #     df = pd.concat([station_ids[i], station_ids[i+1]], ignore_index=True)

    # df.drop_duplicates(subset = 'device_id', ignore_index = True, inplace = True)

    d = {}

    x = len(station_ids)

    for i in range(x):
        curr_df = station_ids[i]
        for j in range(curr_df.shape[0]):
            curr_row = curr_df.iloc[j]
            d_id = curr_row['device_id']
            if d_id in d:
                continue
            else:
                d[d_id] = curr_row

    # print(curr_row)
    df = pd.DataFrame(d)

    return df.T

concat_stations(station_ids)

,device_id,name,longitude,latitude
S77,S77,Alexandra Road,103.8125,1.2937
S109,S109,Ang Mo Kio Avenue 5,103.8492,1.3764
S117,S117,Banyan Road,103.679,1.256
S55,S55,Buangkok Green,103.886,1.3837
S64,S64,Bukit Panjang Road,103.7603,1.3824
...,...,...,...,...
S227,S227,S227,103.80389,1.43944
S228,S228,S228,103.70073,1.34703
S229,S229,S229,103.72195,1.35167
S230,S230,S230,103.76444,1.30167
